In [1]:
from src.datasets.datasets import MovielensDataset
import os
import pandas as pd
import numpy as np

save_dir = os.path.join('data','ml-1m-split')
results_folder = os.path.join('results','mlstm', 'ml-1m-split')

train = MovielensDataset(os.path.join(save_dir,'users_train_dfs.h5'), os.path.join(save_dir,'movies_dfs.h5'), True, True, True, max_history_length=100)
val = MovielensDataset(os.path.join(save_dir,'users_val_dfs.h5'), os.path.join(save_dir,'movies_dfs.h5'), True, True, True, max_history_length=100)
test = MovielensDataset(os.path.join(save_dir,'users_test_dfs.h5'), os.path.join(save_dir,'movies_dfs.h5'), True, True, True, max_history_length=100)

In [11]:
load_dir = os.path.join('data','ml-1m')
ratings_header = ['user_id','movie_id','rating','timestamp']
ratings_df = pd.read_csv(os.path.join(load_dir, 'ratings.dat'),sep='::',names = ratings_header, engine='python')

users_header = ['user_id','gender','age','occupation','zip_code']
users_df = pd.read_csv(os.path.join(load_dir, 'users.dat'),sep='::',names = users_header, engine='python')

movies_header = ['movie_id','title','genres']
movies_df = pd.read_csv(os.path.join(load_dir, 'movies.dat'),sep='::',names = movies_header, encoding='iso-8859-1', engine='python')

In [12]:
len(train), len(val), len(test)

(407929, 45612, 45082)

In [13]:
samples_future_df = pd.read_hdf(os.path.join(save_dir,'users_train_dfs.h5'), key='samples_future')
samples_past_df = pd.read_hdf(os.path.join(save_dir,'users_train_dfs.h5'), key='users_history')

all_samples = pd.concat([samples_future_df, samples_past_df])

movies_df = pd.read_hdf(os.path.join(save_dir,'movies_dfs.h5'), key='movies', mode='r')

In [14]:
movie_averages = all_samples.groupby('movie_id').mean().rating.to_dict()

In [15]:
samples_future_df_test = pd.read_hdf(os.path.join(save_dir,'users_test_dfs.h5'), key='samples_future')

In [16]:
from sklearn.metrics import mean_squared_error

mean_squared_error(samples_future_df_test.rating.to_numpy().astype(np.float32), samples_future_df_test.movie_id.map(movie_averages).fillna(3.575915494506972).to_numpy())

0.9176473052976801

In [17]:
samples_future_df.rating.std(), samples_future_df.rating.mean()

(1.1249758566467858, 3.4905363433342567)

In [18]:
all_samples.rating.std() ** 2, all_samples.rating.mean()

(1.257385221426483, 3.575915494506972)

In [22]:
movies_df = pd.read_hdf(os.path.join(save_dir,'movies_dfs.h5'), key='movies', mode='r')


In [23]:
movies_df

title genres                      \
                                       title Action Adventure Animation   
movie_id                                                                  
1                           Toy Story (1995)    0.0       0.0       1.0   
2                             Jumanji (1995)    0.0       1.0       0.0   
3                    Grumpier Old Men (1995)    0.0       0.0       0.0   
4                   Waiting to Exhale (1995)    0.0       0.0       0.0   
5         Father of the Bride Part II (1995)    0.0       0.0       0.0   
...                                      ...    ...       ...       ...   
3948                 Meet the Parents (2000)    0.0       0.0       0.0   
3949              Requiem for a Dream (2000)    0.0       0.0       0.0   
3950                        Tigerland (2000)    0.0       0.0       0.0   
3951                 Two Family House (2000)    0.0       0.0       0.0   
3952                   Contender, The (2000)    0.0       0.0       0.0   

                                                                             \
         Children's Comedy Crime Documentary Drama Fantasy Film-Noir Horror   
movie_id                                                                      
1               1.0    1.0   0.0         0.0   0.0     0.0       0.0    0.0   
2               1.0    0.0   0.0         0.0   0.0     1.0       0.0    0.0   
3               0.0    1.0   0.0         0.0   0.0     0.0       0.0    0.0   
4               0.0    1.0   0.0         0.0   1.0     0.0       0.0    0.0   
5               0.0    1.0   0.0         0.0   0.0     0.0       0.0    0.0   
...             ...    ...   ...         ...   ...     ...       ...    ...   
3948            0.0    1.0   0.0         0.0   0.0     0.0       0.0    0.0   
3949            0.0    0.0   0.0         0.0   1.0     0.0       0.0    0.0   
3950            0.0    0.0   0.0         0.0   1.0     0.0       0.0    0.0   
3951            0.0    0.0   0.0         0.0   1.0     0.0       0.0    0.0   
3952            0.0    0.0   0.0         0.0   1.0     0.0       0.0    0.0   

                                                               
         Musical Mystery Romance Sci-Fi Thriller  War Western  
movie_id                                                       
1            0.0     0.0     0.0    0.0      0.0  0.0     0.0  
2            0.0     0.0     0.0    0.0      0.0  0.0     0.0  
3            0.0     0.0     1.0    0.0      0.0  0.0     0.0  
4            0.0     0.0     0.0    0.0      0.0  0.0     0.0  
5            0.0     0.0     0.0    0.0      0.0  0.0     0.0  
...          ...     ...     ...    ...      ...  ...     ...  
3948         0.0     0.0     0.0    0.0      0.0  0.0     0.0  
3949         0.0     0.0     0.0    0.0      0.0  0.0     0.0  
3950         0.0     0.0     0.0    0.0      0.0  0.0     0.0  
3951         0.0     0.0     0.0    0.0      0.0  0.0     0.0  
3952         0.0     0.0     0.0    0.0      1.0  0.0     0.0  

[3883 rows x 19 columns]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
movie_genres = mlb.fit_transform(movies_df.genres.str.split('|').map(set))

movies_df = pd.concat([movies_df.title, pd.DataFrame(movie_genres, index=movies_df.index, columns = mlb.classes_)], keys=movies_df.columns, axis=1)

In [ ]:
movies_df.loc[5][('title', 'title')]

In [ ]:
users_df = pd.concat([pd.read_hdf(os.path.join(save_dir,'users_train_dfs.h5'), key='users', mode='r'),
pd.read_hdf(os.path.join(save_dir,'users_val_dfs.h5'), key='users', mode='r'),
pd.read_hdf(os.path.join(save_dir,'users_test_dfs.h5'), key='users', mode='r')])

In [ ]:
users_df

In [7]:
movies_df['genres'].to_numpy().dtype

dtype('int64')

In [8]:
from sklearn.preprocessing import OneHotEncoder



categories = [i for i in range(21)]

enc  = OneHotEncoder(categories=[categories], sparse=False)

occupations = enc.fit_transform(users_df.occupation.to_numpy().reshape(-1, 1))

names = [f'Occupation: {name}' for name in categories]

occupations_df = pd.DataFrame(occupations, index=users_df.index, columns = names, dtype=np.float32)

users_df = pd.concat([users_df.drop('occupation', axis=1), occupations_df], axis=1)

In [21]:
users_df

,gender,age,zip_code,Occupation: 0,Occupation: 1,Occupation: 2,Occupation: 3,Occupation: 4,Occupation: 5,Occupation: 6,...,Occupation: 11,Occupation: 12,Occupation: 13,Occupation: 14,Occupation: 15,Occupation: 16,Occupation: 17,Occupation: 18,Occupation: 19,Occupation: 20
user_id,,,,,,,,,,,,,,,,,,,,,
4036,M,25,46530,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1045,F,1,77477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3774,M,25,52556,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5004,F,56,90402,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4950,M,25,55421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,M,1,60007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4157,M,50,97321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3315,M,25,78731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
users_df.iloc[0,3]

0.0

In [34]:
movies_df = pd.read_hdf(os.path.join(save_dir,'movies_dfs.h5'), key='movies', mode='r')
movies_df.genres

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,
1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
ratings_df[ratings_df.user_id == 4036].sort_values('timestamp')

,user_id,movie_id,rating,timestamp
671896,4036,2028,5,965507074
671790,4036,3210,4,965507074
671773,4036,1258,5,965507106
671840,4036,3105,3,965507106
671782,4036,1431,3,965507106
...,...,...,...,...
671811,4036,260,5,965509356
671834,4036,1198,5,965509356
671868,4036,318,5,965509644
671888,4036,527,5,965509663
